In [1]:
import cv2
import numpy as np
from tqdm import tqdm
import glob
from matplotlib import pyplot as plt
import os

In [2]:
input_dir = 'motion-detection-frames'
output_dir = 'blob-detection-output'

In [3]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
min_area = 5000  # Area minima per considerare un blob come persona
max_area = 50000 # Area massima per considerare un blob come persona

In [5]:
out = cv2.VideoWriter("blob-detection-output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 15, (3420,1910))

In [6]:
for i in tqdm(range(len(glob.glob1(input_dir, "*.jpg")) - 1)):
    frame = cv2.imread(f"{input_dir}/frame{i}.jpg", cv2.IMREAD_GRAYSCALE)
    # identify connected component (regions with same intensity pixel)
    num_labels, labels_im = cv2.connectedComponents(frame)
    output_frame = cv2.merge([frame, frame, frame])  # Converti in RGB

    for label in range(1, num_labels):
        mask = labels_im == label
        area = np.sum(mask)
        if min_area < area < max_area:
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(output_frame, contours, -1, (0, 255, 0), 2)
    out.write(output_frame)
    cv2.imwrite(f"{output_dir}/frame{i}.jpg", output_frame)    
out.release()

            

  0%|          | 0/88 [00:00<?, ?it/s]

100%|██████████| 88/88 [00:04<00:00, 19.36it/s]


In [5]:
def extract_features(contour):
    area = cv2.contourArea(contour)  # Area del blob
    x, y, w, h = cv2.boundingRect(contour)  # Bounding box
    aspect_ratio = float(h) / w  # Rapporto altezza/larghezza
    perimeter = cv2.arcLength(contour, True)  # Perimetro
    circularity = (4 * np.pi * area) / (perimeter**2) if perimeter > 0 else 0  # Circolarità
    
    return {
        "area": area,
        "aspect_ratio": aspect_ratio,
        "circularity": circularity,
        "bounding_box": (x, y, w, h)
    }

In [6]:
features_list = []

In [ ]:
for i in tqdm(range(len(glob.glob1(input_dir, "*.jpg")) - 1)):
    frame = cv2.imread(f"{input_dir}/frame{i}.jpg", cv2.IMREAD_GRAYSCALE)
    contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if min_area < area < max_area:
            features = extract_features(contour)
            features["frame"] = f"frame{i}.jpg"
            features_list.append(features)
            
            x, y, w, h = features["bounding_box"]
            output_frame = cv2.merge([frame, frame, frame])  # Converti in RGB
            cv2.rectangle(output_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.imwrite(f"{output_dir}/frame{i}.jpg", output_frame)    

100%|██████████| 748/748 [00:04<00:00, 153.77it/s]


In [ ]:
frame = cv2.imread(f"{input_dir}/frame19.jpg", cv2.IMREAD_GRAYSCALE)
contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    area = cv2.contourArea(contour)
    if min_area < area < max_area:
        features = extract_features(contour)
        features["frame"] = f"frame{i}.jpg"
        features_list.append(features)
        
        x, y, w, h = features["bounding_box"]
        output_frame = cv2.merge([frame, frame, frame])  # Converti in RGB
        cv2.rectangle(output_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        plt.imshow(output_frame)
        plt.show()

2024-12-10 15:21:31.968 Python[48549:1909095] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-10 15:21:31.968 Python[48549:1909095] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 

In [13]:
print(features_list)

[{'area': 35338.5, 'aspect_ratio': 0.6857142857142857, 'circularity': 0.00825390601511893, 'bounding_box': (288, 424, 455, 312), 'frame': 'frame19.jpg'}, {'area': 39292.0, 'aspect_ratio': 0.6551724137931034, 'circularity': 0.010494407515974278, 'bounding_box': (672, 368, 464, 304), 'frame': 'frame19.jpg'}, {'area': 9916.5, 'aspect_ratio': 0.9583333333333334, 'circularity': 0.022131939905631955, 'bounding_box': (320, 552, 192, 184), 'frame': 'frame20.jpg'}, {'area': 11194.0, 'aspect_ratio': 0.3617021276595745, 'circularity': 0.0066208023903454385, 'bounding_box': (376, 416, 376, 136), 'frame': 'frame20.jpg'}, {'area': 23662.0, 'aspect_ratio': 0.6363636363636364, 'circularity': 0.006597587881362886, 'bounding_box': (712, 360, 440, 280), 'frame': 'frame20.jpg'}, {'area': 9730.0, 'aspect_ratio': 1.170731707317073, 'circularity': 0.027648747453829815, 'bounding_box': (416, 528, 164, 192), 'frame': 'frame23.jpg'}, {'area': 10966.0, 'aspect_ratio': 0.5673076923076923, 'circularity': 0.0313275